# AI‑Powered Climate Risk Travel Planner

**A detailed internship-ready project notebook**

**Contents:** project overview, objectives, architecture, data schema, ML prototype (synthetic), API skeletons, deployment notes, milestones, and deliverables.

**Author:** Internship Candidate

**Date:** 2025-09-07

## 1. One-line summary
A web/mobile app that gives travelers an AI-driven **risk score** (0–100) for a destination/date, real-time disaster alerts, alternatives/recommendations, and emergency support info.

## 2. Objectives (measurable)
- Short-term travel risk score (0–100) for destination/date.
- Real-time alerts for ongoing disasters.
- Recommend 3 actionable options: keep, reschedule, alternative.
- Provide emergency contacts and guidance.
- Deliver working prototype (frontend + backend + ML) by internship end.

## 3. High-level architecture
1. Frontend: React / React Native UI
2. Backend: FastAPI (recommended) or Flask
3. Ingest: weather APIs, disaster feeds (GDACS/NDMA), satellite indices (optional)
4. DB / Feature store: PostgreSQL or MongoDB
5. ML module: model server returning risk_score & explanation
6. Notifications: Push/Twilio/email
7. Monitoring: simple logs / Prometheus & Grafana

## 4. Data schema (recommended columns)
`date, lat, lon, temp_c, precip_24h_mm, precip_72h_mm, wind_kmph, humidity_pct, aqi, river_level_m, sea_level_pressure_hpa, soil_moisture, historical_flood_count_5yr, elevation_m, landcover_type, is_cyclone_warning, is_excess_rain_advisory, label_risk`

## 5. Feature ideas & sources
- Precipitation (forecast & recent accumulation) — OpenWeatherMap, IMD
- Wind gusts — weather APIs
- Wind gusts — weather APIs
- River gauge levels — local hydrology APIs (if available)
- AQI — local monitoring networks
- Satellite flood index — Sentinel-derived indices (optional)
- Historical disaster frequency — EM-DAT, local records
- Elevation & landcover — SRTM / global landcover datasets

## 6. Baseline ML prototype (synthetic data)
The cell below trains a RandomForest baseline on synthetic features, evaluates it, and saves a small sample CSV. Replace synthetic data with real features when available.

In [ ]:
# Baseline ML prototype using synthetic data
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import os

np.random.seed(42)
N = 1500
df = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=N, freq='H').strftime('%Y-%m-%d %H:%M:%S'),
    'lat': np.random.uniform(-90, 90, N),
    'lon': np.random.uniform(-180, 180, N),
    'temp_c': np.random.normal(25, 8, N),
    'precip_24h_mm': np.random.exponential(5, N),
    'precip_72h_mm': np.random.exponential(12, N),
    'wind_kmph': np.random.normal(15, 10, N).clip(0),
    'humidity_pct': np.random.uniform(30, 100, N),
    'aqi': np.random.uniform(10, 400, N),
    'river_level_m': np.random.normal(2, 1, N).clip(0),
    'sea_level_pressure_hpa': np.random.normal(1013, 10, N),
    'elevation_m': np.random.uniform(0, 3000, N)
})

df['label_risk'] = 0
df.loc[(df['precip_72h_mm'] > 50) | (df['wind_kmph'] > 70) | (df['river_level_m'] > 6), 'label_risk'] = 2
df.loc[((df['precip_72h_mm'] > 20) & (df['precip_24h_mm'] > 10)) | (df['wind_kmph'] > 40), 'label_risk'] = 1

features = ['temp_c','precip_24h_mm','precip_72h_mm','wind_kmph','humidity_pct','aqi','river_level_m','sea_level_pressure_hpa','elevation_m']
X = df[features]
y = df['label_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('Classification report:\n')
print(classification_report(y_test, y_pred))
print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))

importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print('\nFeature importances:\n', importances)

os.makedirs('/mnt/data/model_output', exist_ok=True)
joblib.dump(model, '/mnt/data/model_output/risk_model_rf.pkl')
X_test.head(50).to_csv('/mnt/data/model_output/sample_test_features.csv', index=False)
print('\nSaved model and sample CSV to /mnt/data/model_output/')

## 7. Simple explainability
Use feature importances above now. For per-prediction explanations, integrate SHAP or LIME when packaging the final model.

## 8. API skeletons (FastAPI / Flask)
Use the skeletons in code cells to start your backend. Replace placeholder features with real features computed from API pulls.

In [ ]:
# FastAPI skeleton (example)
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

app = FastAPI()
try:
    model = joblib.load('/mnt/data/model_output/risk_model_rf.pkl')
except:
    model = None

class RiskRequest(BaseModel):
    lat: float
    lon: float
    start_date: str
    end_date: str
    travelers: int = 1
    special_needs: list = []

@app.post('/risk')
def get_risk(req: RiskRequest):
    # Replace the following with real feature extraction
    features = np.array([[25.0, 10.0, 30.0, 15.0, 70.0, 100.0, 2.0, 1012.0, 50.0]])
    if model is not None:
        pred = model.predict_proba(features)
        risk_score = float(pred[0].dot(np.arange(len(pred[0])))/ (len(pred[0])-1) * 100) if pred.shape[1]>1 else float(pred[0].max()*100)
        risk_level = 'High' if risk_score>66 else ('Medium' if risk_score>33 else 'Low')
    else:
        risk_score = 42.0
        risk_level = 'Medium'
    return {'risk_score': risk_score, 'risk_level': risk_level}

In [ ]:
# Flask skeleton (example)
from flask import Flask, request, jsonify
import joblib, numpy as np
app = Flask(__name__)
try:
    model = joblib.load('/mnt/data/model_output/risk_model_rf.pkl')
except:
    model = None

@app.route('/risk', methods=['POST'])
def risk():
    data = request.json or {}
    features = np.array([[25.0, 5.0, 12.0, 10.0, 60.0, 80.0, 1.5, 1010.0, 100.0]])
    if model is not None:
        pred = model.predict_proba(features)
        risk_score = float(pred[0].dot(np.arange(len(pred[0])))/ (len(pred[0])-1) * 100) if pred.shape[1]>1 else float(pred[0].max()*100)
        risk_level = 'High' if risk_score>66 else ('Medium' if risk_score>33 else 'Low')
    else:
        risk_score = 25.0
        risk_level = 'Low'
    return jsonify({'risk_score': risk_score, 'risk_level': risk_level})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

## 9. Notifications & alerting
- Scheduler (APScheduler/Celery) to poll feeds and compute user-specific alerts.
- Use Twilio / Firebase Cloud Messaging for SMS/push.
- Always include official feed links and confidence scores in alerts.

## 10. Evaluation & metrics
- Class metrics (precision/recall/F1), calibration (Brier), historical-event matching. Include user feedback for continual improvement.

## 11. 8-week milestone plan
Week 1: Setup & mockups
Week 2: Ingest & DB
Week 3: Baseline model
Week 4: Backend & /risk
Week 5: Emergency info & alerts
Week 6: Explainability & refine
Week 7: Frontend polish
Week 8: Finalize & demo

## 12. Deliverables
- Working demo, notebooks, API endpoints, README, slides, optional demo video.

In [ ]:
# Small README file written alongside the notebook
readme = """AI-Powered Climate Risk Travel Planner
Files to look for after running cells:
- /mnt/data/model_output/risk_model_rf.pkl
- /mnt/data/model_output/sample_test_features.csv
Run the ML prototype cell to generate these files.
""
open('/mnt/data/AI_Powered_Climate_Risk_Travel_Planner_README.txt','w').write(readme)
print('Wrote README to /mnt/data/AI_Powered_Climate_Risk_Travel_Planner_README.txt')